# Isomatrix Tools

> Tools for converting isomatrix files into anndata objects for integration with the Scanpy ecosystem.


In [1]:
#| default_exp isomatrix_tools

In [2]:
#| hide
from nbdev.showdoc import *

In [3]:
#| export
import pandas as pd
import scanpy as sc
from scanpy import AnnData
from scipy.sparse import csr_matrix
import warnings

def isomatrix_to_anndata(file_path:str,  # The path to the isomatrix csv file to be read.
                        sparse:bool=True  # Flag to determine if the output should be a sparse matrix.
) -> AnnData: # The converted isomatrix as a scanpy compatible  anndata object
    """
    This function converts an isomatrix txt file (SiCeLoRe output) into an AnnData object compatible with scanpy

    """
    
    # Read in the data from the file
    df = pd.read_csv(file_path, sep='\t', index_col=0)
    # Filter out rows where the transcriptId is "undef"
    df = df.loc[df["transcriptId"] != "undef"]
    
    df = df.reset_index()
    df = df.transpose()
    
    # Extract the rows with 'gene_id', 'transcript_id', 'nb_exons' from the DataFrame
    additional_info_rows = df.loc[df.index.intersection(['geneId', 'transcriptId', 'nbExons'])]
    # Drop 'gene_id', 'transcript_id', 'nb_exons' rows from the DataFrame if they exist
    df = df.drop(['geneId', 'transcriptId', 'nbExons'], errors='ignore')

    # Convert the DataFrame to a sparse matrix if the sparse flag is True
    if sparse:
        matrix = csr_matrix(df.values.astype('float32'))
    else:
        try:
            matrix = df.values.astype('float32')
        except ValueError:
            print("Error: Non-numeric data present in the DataFrame. Cannot convert to float.")
            return None
    
    # Convert the matrix to an AnnData object
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        anndata = sc.AnnData(X=matrix, obs=pd.DataFrame(index=df.index), var=pd.DataFrame(index=df.columns))
    
    # Add additional information to the AnnData object vars
    for info in ['geneId', 'transcriptId', 'nbExons']:
        if info in additional_info_rows.index:
            anndata.var[info] = additional_info_rows.loc[info, :].values
            if info == 'nbExons':
                anndata.var[info] = anndata.var[info].astype('int32')
    
    return anndata

In [4]:
#| export
def download_test_data() -> str: #The absolute path of the extracted file 'sample_isomatrix.txt' if the download is successful.
    """
    This function downloads a test data file from a specified URL, saves it locally, and extracts it.
    """
    import urllib.request
    import gzip
    import shutil
    import os

    # URL of the file to be downloaded
    url = "https://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3748nnn/GSM3748087/suppl/GSM3748087%5F190c.isoforms.matrix.txt.gz"

    # Download the file from `url` and save it locally under `file.txt.gz`:
    urllib.request.urlretrieve(url, 'file.txt.gz')

    # Check if the file is downloaded correctly
    if os.path.exists('file.txt.gz'):
        print("File downloaded successfully")
        # Now we need to extract the file
        with gzip.open('file.txt.gz', 'rb') as f_in:
            with open('sample_isomatrix.txt', 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
        print("File extracted successfully")
        return os.path.abspath('sample_isomatrix.txt')
    else:
        print("Failed to download the file")
        return None


Example usage of `isomatrix_to_anndata`: We can use the `download_test_data` function to download a small isoform matrix dataset for demonstrating the functionality.


In [5]:
from longreadtools.isomatrix_tools import * 
test_file = download_test_data() 

File downloaded successfully
File extracted successfully


In [6]:
anndata = isomatrix_to_anndata(test_file)

Lets take a look at the anndata object generated from the isomatrix.

In [7]:
anndata.var

,geneId,transcriptId
0,Klc2,ENSMUST00000156717.1
1,Capn15,ENSMUST00000212520.1
2,Klc2,ENSMUST00000025798.12
3,Eva1c,ENSMUST00000231280.1
4,Atg5,ENSMUST00000039286.4
...,...,...
20829,Kcnj9,ENSMUST00000062387.7
20830,Iqcg,ENSMUST00000115100.8
20831,Nt5dc2,ENSMUST00000227096.1
20832,Emg1,ENSMUST00000004379.7


In [8]:
#| hide
def test_isomatrix_to_anndata():
    # Test with a known file
    test_file = download_test_data()
    anndata = isomatrix_to_anndata(test_file)

    # Check the type of the returned object
    assert isinstance(anndata, sc.AnnData), "The returned object is not an AnnData object."

    # Check the dimensions of the AnnData object
    assert anndata.shape == (190, 20834), "The dimensions of the AnnData object are not as expected."

    # Check the var names of the AnnData object
    assert 'geneId' in anndata.var, "The 'geneId' is not in the var of the AnnData object."
    assert 'transcriptId' in anndata.var, "The 'transcriptId' is not in the var of the AnnData object."




In [9]:
test_file = download_test_data()

File downloaded successfully
File extracted successfully


Often, it may be necessary to convert more than one isomatrix in bulk. The function `multiple_isomatrix_conversion` has been designed for this purpose. It leverages Python's multiprocessing capabilities to perform this task in a fast and efficient manner.


In [10]:
#| export
import numpy as np 
from pandas import DataFrame

def simulate_isomatrix(num_genes: int, # number of genes (groups of rows)
                       num_transcripts_per_gene: int, # number of transcripts per gene
                       num_samples: int, # number of samples (columns)
                       sparsity: float = 0.95, # fraction of zeros in the data (default 0.95)
                       max_expression: int = 100, # maximum expression level for any transcript in any sample
                       seed: int = 0 # random seed for reproducibility
                      ) -> DataFrame : # DataFrame with simulated transcript expression data for demonstration purposes.
    """
    Simulate transcript expression data to match the structure of the first image provided by the user.
    Allows specifying the number of genes, transcripts per gene, and samples.
    """
    # Set random seed for reproducibility
    np.random.seed(seed)
    
    # Calculate total number of transcripts
    total_transcripts = num_genes * num_transcripts_per_gene
    
    # Generate random data
    data = np.random.rand(total_transcripts, num_samples)
    
    # Apply sparsity
    zero_mask = np.random.rand(total_transcripts, num_samples) > sparsity
    data[~zero_mask] = 0  # Set a fraction of data to 0 based on sparsity
    
    # Scale data to have values up to max_expression
    data = np.ceil(data * max_expression).astype(int)
    
    # Generate transcript and sample labels
    transcript_ids = [f"ENSMUST00000{str(i).zfill(6)}_{str(j).zfill(6)}_{str(seed).zfill(6)}.1" for j in range(num_genes) for i in range(1, num_transcripts_per_gene + 1)]
    gene_ids = [f"Gene_{(i // num_transcripts_per_gene) + 1}" for i in range(total_transcripts)]
    nb_exons = np.random.randint(1, 21, total_transcripts)  # Assuming 1-20 exons based on typical gene structures
    sample_ids = [f"CACCTACACGTCAAC{str(i).zfill(2)}" for i in range(1, num_samples + 1)]
    
    # Create DataFrame
    df = pd.DataFrame(data, index=gene_ids, columns=sample_ids)
    df.index.name = 'geneId'  # Add index name
    df.insert(0, 'transcriptId', transcript_ids)
    df.insert(1, 'nbExons', nb_exons)
    
    return df


In [11]:
#| export
import os
def simulate_and_save_isomatrices(num_isomatrix: int, #number of isomatrix to generate
                                num_genes: int, # number of genes (groups of rows)
                                num_transcripts_per_gene: int, # number of transcripts per gene
                                num_samples: int, # number of samples (columns)
                                sparsity: float = 0.95, # fraction of zeros in the data (default 0.95)
                                max_expression: int = 100, # maximum expression level for any transcript in any sample
                                seed: int = 0, # random seed for reproducibility
                                output_dir: str = './', # directory to save the generated isomatrix txt files
                                return_paths: bool = False, # return paths to the isomatrixs as a list of strings if True
                                verbose: bool = False # print progress messages if True
                               ) -> list: # list of paths for the simulated matrices (if return is set True)
    
    """
    Simulate multiple isomatrix and save them as txt files in the specified directory.
    If return_paths is True, return a list of paths to the saved isomatrix files.
    """
    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)
    
    output_files = []
    for i in range(num_isomatrix):
        # Generate isomatrix
        df = simulate_isomatrix(num_genes, num_transcripts_per_gene, num_samples, sparsity, max_expression, seed+i)
        
        # Save to txt file
        output_file = os.path.join(output_dir, f'isomatrix_{i+1}.txt')
        df.to_csv(output_file, sep='\t')
        
        if verbose:
            print(f'Isomatrix {i+1} saved to {output_file}')
        output_files.append(output_file)
    
    if return_paths:
        return output_files

In [12]:
#| hide
#| export
import os
import time

def convert_and_save_file(sample, verbose, sparse=False):
    anndata = isomatrix_to_anndata(sample, sparse=sparse)
    h5ad_file = sample.replace('.txt', '.h5ad')
    
    # Check if the file already exists and delete it if it does
    if os.path.exists(h5ad_file):
        os.remove(h5ad_file)
    
    # Add a delay and retry mechanism to handle file locking issues
    for attempt in range(10):
        try:
            anndata.write_h5ad(h5ad_file)
            break
        except BlockingIOError:
            if attempt < 9:  # i.e. if this is not the last attempt
                time.sleep(1)  # wait for 1 second before the next attempt
            else:
                raise  # re-raise the last exception if all attempts fail
    
    if verbose:
        print(f"File {h5ad_file} was successfully written to disk.")
    
    return h5ad_file




In [13]:
#| export
from multiprocessing import Pool
import os
from functools import partial


def multiple_isomatrix_conversion(file_paths: list, # A list of file paths to be converted.
                                  verbose: bool = False, # If True, print progress messages.
                                  return_paths: bool = False, # If True, return a list of paths to the converted files.
                                  sparse: bool = False # If True, the anndata object will be stored in sparse format.
                                  ) -> list: # A list of paths to the converted files.
    """
    This function takes a list of file paths, converts each file from isomatrix to anndata format, 
    and saves the converted file in the same location with the same name but with a .h5ad extension.
    If return_paths is True, it returns a list of paths to the converted files.
    If sparse is True, the anndata object will be stored in sparse format.
    """
    with Pool() as p:
        converted_files = p.map(partial(convert_and_save_file, verbose=verbose, sparse=sparse), file_paths)
    
    if return_paths:
        return converted_files

Here is an example of how to use the function to convert multiple Isomatrix objects simultaneously. The function requires a list of paths to the Isomatrix text files as input. To demonstrate this functionality and to aid further development by other contributors, the ability to simulate the Isomatrix data has been provided.
 
In this section, we will be making use of the `simulate_and_save_isomatrices` function to generate and store 10 isomatrices. Following this, the `multiple_isomatrix_conversion` function will be employed to convert these isomatrices into anndata objects and subsequently save them to the disk.



In [14]:
#| hide

# import os
# import re

# directory = '/data/analysis/data_mcandrew/000-sclr-discovair/'

# # Define the regular expression pattern
# pattern = re.compile('.*(_BIOP_INT|BIOP_NAS)$')

# # Get a list of all files in the directory
# all_files = os.listdir(directory)

# # Filter the list to include only files that match the pattern
# matching_files = [os.path.join(directory, f) for f in all_files if pattern.match(f)]

# # Print the list of matching files
# print(matching_files)

# individual_runs = matching_files
# individual_runs = [f'{run}_isomatrix.txt' for run in individual_runs]
# individual_runs = [os.path.join(run, f'{os.path.basename(run)}_isomatrix.txt') for run in matching_files]

# isomatrice_paths = individual_runs
# anndata_paths = multiple_isomatrix_conversion(isomatrice_paths, verbose=True, return_paths=True)

In [15]:
#| hide
#| export
from joblib import Parallel, delayed
from tqdm import tqdm

def load_and_set_var_names(path):
    dataset = sc.read_h5ad(path, backed='r')  # Read the file in 'backed' mode to avoid loading the whole data into memory
    dataset.var_names = dataset.var['transcriptId']
    return dataset

Now, let's simulate and convert multiple isomatrices to showcase the functionality of our concatenation process. 

In [16]:
simulation = simulate_and_save_isomatrices(num_isomatrix=10, num_genes=1000, num_transcripts_per_gene=5, num_samples=100, sparsity=0.95, max_expression=100, seed=0, output_dir='./', return_paths=True, verbose=True)
anndata_paths = multiple_isomatrix_conversion(simulation, verbose=True, return_paths=True)

Isomatrix 1 saved to ./isomatrix_1.txt
Isomatrix 2 saved to ./isomatrix_2.txt
Isomatrix 3 saved to ./isomatrix_3.txt
Isomatrix 4 saved to ./isomatrix_4.txt
Isomatrix 5 saved to ./isomatrix_5.txt
Isomatrix 6 saved to ./isomatrix_6.txt
Isomatrix 7 saved to ./isomatrix_7.txt
Isomatrix 8 saved to ./isomatrix_8.txt
Isomatrix 9 saved to ./isomatrix_9.txt
Isomatrix 10 saved to ./isomatrix_10.txt
File ./isomatrix_5.h5ad was successfully written to disk.File ./isomatrix_2.h5ad was successfully written to disk.

File ./isomatrix_3.h5ad was successfully written to disk.
File ./isomatrix_7.h5ad was successfully written to disk.
File ./isomatrix_8.h5ad was successfully written to disk.File ./isomatrix_10.h5ad was successfully written to disk.

File ./isomatrix_6.h5ad was successfully written to disk.
File ./isomatrix_1.h5ad was successfully written to disk.
File ./isomatrix_9.h5ad was successfully written to disk.
File ./isomatrix_4.h5ad was successfully written to disk.


Here we have two other useful tools for working with isomatrix anndata. The first, `feature_set_standardization`, can take a list of isomatrix anndata objects and standardize the features. For instance, if we run the function with `standardization_method` set to 'union', it will return a list of anndata objects with the .var attribute being the union of all geneId, transcriptId, and nbExons. Any values missing in the original datasets will be set to zero. Alternatively, if we run it with `standardization_method` set to 'intersection', only the intersecting features will be retained.



In [17]:
#| export
def feature_set_standardization(adatas:list, # list of AnnData objects or paths to AnnData files
                                standardization_method:str = 'union' # str specifiying method to use union or intersection
                                ) -> list: # list of anndata objects with the features standardised 
    """
    Standardize the feature sets of multiple AnnData objects.
    
    This function takes a list of AnnData objects or paths to AnnData files and a standardization method as input.
    The standardization method can be either 'union' or 'intersection'.
    If 'union' is chosen, the function will create a union of all features across all AnnData objects.
    If 'intersection' is chosen, the function will create an intersection of all features across all AnnData objects.
    The function returns a list of standardized AnnData objects.
    """
    # Check if the first element in adatas is a string
    if isinstance(adatas[0], str):
        # If it is, load anndata objects from paths
        adatas = [load_and_set_var_names(path) for path in adatas]

    all_features = set()
    common_features = set()
    
    # Get union or intersection of all feature sets across all AnnData objects
    if standardization_method == 'union':
        all_features = set().union(*[set(adata.var.itertuples(index=False, name=None)) for adata in adatas])
    elif standardization_method == 'intersection':
        common_features = set.intersection(*[set(adata.var['transcriptId']) for adata in adatas])
    else:
        raise ValueError("standardization_method should be 'union' or 'intersection'")
    

    standardized_adatas = []
    # Iterate over each AnnData object
    for dataset in tqdm(adatas, desc= f"Standardizing anndata features via {standardization_method}"):
        dataset.var_names = dataset.var['transcriptId']
        existing_var = dataset.var
        # Identify features that are in the union/intersection but not in the current AnnData object
        missing_features = all_features - set(dataset.var.itertuples(index=False, name=None))
        if standardization_method == 'union':
            if missing_features:
                # Create a DataFrame of zeros with rows equal to the number of observations in the current AnnData object
                # and columns equal to the number of missing features
                zero_data = np.zeros((dataset.n_obs, len(missing_features)), dtype=object)
                zero_df = pd.DataFrame(zero_data, index=dataset.obs_names, columns=pd.Index([t[1] for t in missing_features], name='transcriptId'))

                # Merge the zero_df with the dataset's .to_df() DataFrame along the columns
                combined_df = pd.concat([dataset.to_df(), zero_df], axis=1)

                # Convert the combined DataFrame back to an AnnData object
                combined_data = sc.AnnData(combined_df, obs=dataset.obs, var=pd.DataFrame(index=combined_df.columns))

                missing_df = pd.DataFrame(list(missing_features), columns=['geneId', 'transcriptId', 'nbExons'])
                missing_df.set_index('transcriptId', inplace=True)

                combined_data.var = pd.concat([existing_var, missing_df], axis=0)
                combined_data.var['transcriptId'] = combined_data.var_names
                standardized_adatas.append(combined_data)
            else:
                # If no features are missing, append the dataset as is
                standardized_adatas.append(dataset)
        elif standardization_method == 'intersection':
            # Subset the dataset to only include transcriptIds that are in the intersection
            dataset = dataset[:, dataset.var_names.isin(common_features)]
            standardized_adatas.append(dataset)
    return standardized_adatas


Lets run the function on are simulated data and see what the result looks like.

In [18]:
feature_set_standardization(anndata_paths)

Standardizing anndata features via union:   0%|          | 0/10 [00:00<?, ?it/s]/home/mcandrew/.conda/envs/scLRanalyis/lib/python3.11/site-packages/anndata/utils.py:292: UserWarning: X converted to numpy array with dtype object
  warnings.warn(f"{name} converted to numpy array with dtype {arr.dtype}")
Standardizing anndata features via union:  10%|█         | 1/10 [00:00<00:05,  1.74it/s]/home/mcandrew/.conda/envs/scLRanalyis/lib/python3.11/site-packages/anndata/utils.py:292: UserWarning: X converted to numpy array with dtype object
  warnings.warn(f"{name} converted to numpy array with dtype {arr.dtype}")
Standardizing anndata features via union:  20%|██        | 2/10 [00:01<00:04,  1.76it/s]/home/mcandrew/.conda/envs/scLRanalyis/lib/python3.11/site-packages/anndata/utils.py:292: UserWarning: X converted to numpy array with dtype object
  warnings.warn(f"{name} converted to numpy array with dtype {arr.dtype}")
Standardizing anndata features via union:  30%|███       | 3/10 [00:01<00:0

[AnnData object with n_obs × n_vars = 100 × 50000
     var: 'geneId', 'transcriptId', 'nbExons',
 AnnData object with n_obs × n_vars = 100 × 50000
     var: 'geneId', 'transcriptId', 'nbExons',
 AnnData object with n_obs × n_vars = 100 × 50000
     var: 'geneId', 'transcriptId', 'nbExons',
 AnnData object with n_obs × n_vars = 100 × 50000
     var: 'geneId', 'transcriptId', 'nbExons',
 AnnData object with n_obs × n_vars = 100 × 50000
     var: 'geneId', 'transcriptId', 'nbExons',
 AnnData object with n_obs × n_vars = 100 × 50000
     var: 'geneId', 'transcriptId', 'nbExons',
 AnnData object with n_obs × n_vars = 100 × 50000
     var: 'geneId', 'transcriptId', 'nbExons',
 AnnData object with n_obs × n_vars = 100 × 50000
     var: 'geneId', 'transcriptId', 'nbExons',
 AnnData object with n_obs × n_vars = 100 × 50000
     var: 'geneId', 'transcriptId', 'nbExons',
 AnnData object with n_obs × n_vars = 100 × 50000
     var: 'geneId', 'transcriptId', 'nbExons']

In [19]:
#| export
import anndata as ad
from scipy.sparse import csr_matrix

def concatenate_anndata(h5ad_inputs: list, # A list of AnnData objects or paths to .h5ad files.
                         standardization_method='union', # The method to standardize the feature sets across all AnnData objects. It can be either 'union' or 'intersection'. Default is 'union'.
                         sparse=False # Optional flag to convert the final matrix to sparse. Default is False.
                         ) -> AnnData: # The concatenated AnnData object.
    """
    This function concatenates multiple AnnData objects into a single AnnData object.
    """
    
    # Check if inputs are paths or actual anndata objects
    if isinstance(h5ad_inputs[0], str):
        # If inputs are paths, read the .h5ad files and generate batch keys based on the directory names
        to_concat = [sc.read_h5ad(input, backed='r') for input in h5ad_inputs]
        batch_keys = [os.path.basename(os.path.dirname(input)) for input in h5ad_inputs]
    else:
        # If inputs are AnnData objects, use them directly and generate unique batch keys for each dataset
        to_concat = h5ad_inputs
        batch_keys = [f"batch_{i}" for i, adata in enumerate(h5ad_inputs)]

    # Apply feature set standardization
    to_concat = feature_set_standardization(to_concat, standardization_method)

    # Ensure unique keys for concatenation
    if len(batch_keys) != len(set(batch_keys)):
        # If batch keys are not unique, create new unique batch keys
        batch_keys = [f"batch_{i}" for i in range(len(to_concat))]

    # Concatenate anndata objects
    concat_anndata = ad.concat(
        to_concat,
        join="outer",
        label='batch',
        keys=batch_keys,
        index_unique=None  # This will speed up the concatenation process by not checking for unique indices
    )

    # Set the .var attribute of the concatenated AnnData object to be the same as the first input AnnData object
    concat_anndata.var = to_concat[0].var

    # If the sparse flag is True, convert the final matrix to sparse
    if sparse:
        concat_anndata.X = csr_matrix(concat_anndata.X)

    return concat_anndata


Now, we will showcase the usage of our `concatenate_anndata` function. This function incorporates the `feature_set_standardization` function. The expected output is a correctly configured, concatenated isomatrix anndata object, ready for subsequent analysis.


In [20]:
andata_concat = concatenate_anndata(anndata_paths)

Standardizing anndata features via union:   0%|          | 0/10 [00:00<?, ?it/s]/home/mcandrew/.conda/envs/scLRanalyis/lib/python3.11/site-packages/anndata/utils.py:292: UserWarning: X converted to numpy array with dtype object
  warnings.warn(f"{name} converted to numpy array with dtype {arr.dtype}")
Standardizing anndata features via union:  10%|█         | 1/10 [00:00<00:04,  2.12it/s]/home/mcandrew/.conda/envs/scLRanalyis/lib/python3.11/site-packages/anndata/utils.py:292: UserWarning: X converted to numpy array with dtype object
  warnings.warn(f"{name} converted to numpy array with dtype {arr.dtype}")
Standardizing anndata features via union:  20%|██        | 2/10 [00:01<00:04,  1.87it/s]/home/mcandrew/.conda/envs/scLRanalyis/lib/python3.11/site-packages/anndata/utils.py:292: UserWarning: X converted to numpy array with dtype object
  warnings.warn(f"{name} converted to numpy array with dtype {arr.dtype}")
Standardizing anndata features via union:  30%|███       | 3/10 [00:01<00:0

In [21]:
andata_concat

AnnData object with n_obs × n_vars = 1000 × 50000
    obs: 'batch'
    var: 'geneId', 'transcriptId', 'nbExons'

In [22]:
andata_concat.var

,geneId,transcriptId,nbExons
transcriptId,,,
ENSMUST00000000001_000000_000000.1,Gene_1,ENSMUST00000000001_000000_000000.1,10
ENSMUST00000000002_000000_000000.1,Gene_1,ENSMUST00000000002_000000_000000.1,6
ENSMUST00000000003_000000_000000.1,Gene_1,ENSMUST00000000003_000000_000000.1,18
ENSMUST00000000004_000000_000000.1,Gene_1,ENSMUST00000000004_000000_000000.1,17
ENSMUST00000000005_000000_000000.1,Gene_1,ENSMUST00000000005_000000_000000.1,9
...,...,...,...
ENSMUST00000000004_000690_000008.1,Gene_691,ENSMUST00000000004_000690_000008.1,13
ENSMUST00000000004_000064_000003.1,Gene_65,ENSMUST00000000004_000064_000003.1,4
ENSMUST00000000003_000096_000009.1,Gene_97,ENSMUST00000000003_000096_000009.1,4


In [23]:
#| hide
import nbdev; nbdev.nbdev_export()